In [2]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import nltk

nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df = pd.read_excel('IMDB_Dataset_sample.xlsx')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
def preprocess_text(text):

    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    text = text.lower()
    text = text.strip()
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = stopwords.words("english")
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens


df['tokens'] = df['review'].apply(preprocess_text)
df

,review,sentiment,tokens
0,One of the other reviewers has mentioned that ...,positive,"[one, reviewers, mentioned, watching, oz, epis..."
1,A wonderful little production. <br /><br />The...,positive,"[wonderful, little, production, br, br, filmin..."
2,I thought this was a wonderful way to spend ti...,positive,"[thought, wonderful, way, spend, time, hot, su..."
3,Basically there's a family where a little boy ...,negative,"[basically, theres, family, little, boy, jake,..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[petter, matteis, love, time, money, visually,..."
...,...,...,...
1109,I have not read the book that this was based u...,positive,"[read, book, based, uponinspired, ofthe, other..."
1110,"Brilliant thriller, deserving far more fame, M...",positive,"[brilliant, thriller, deserving, far, fame, mi..."
1111,I saw this movie a fews years ago and was lite...,positive,"[saw, movie, fews, years, ago, literally, swep..."
1112,I just watched this movie on Bravo! and it was...,negative,"[watched, movie, bravo, absolutely, horrible, ..."


In [6]:
model = Word2Vec(sentences=df['tokens'],vector_size=100,window=5,min_count=1,sg=1)

In [7]:
vl = list(model.wv.index_to_key)

print(len(vl))

21781


In [8]:
df['tokens'][0]

['one',
 'reviewers',
 'mentioned',
 'watching',
 'oz',
 'episode',
 'youll',
 'hooked',
 'right',
 'exactly',
 'happened',
 'mebr',
 'br',
 'first',
 'thing',
 'struck',
 'oz',
 'brutality',
 'unflinching',
 'scenes',
 'violence',
 'set',
 'right',
 'word',
 'go',
 'trust',
 'show',
 'faint',
 'hearted',
 'timid',
 'show',
 'pulls',
 'punches',
 'regards',
 'drugs',
 'sex',
 'violence',
 'hardcore',
 'classic',
 'use',
 'wordbr',
 'br',
 'called',
 'oz',
 'nickname',
 'given',
 'oswald',
 'maximum',
 'security',
 'state',
 'penitentary',
 'focuses',
 'mainly',
 'emerald',
 'city',
 'experimental',
 'section',
 'prison',
 'cells',
 'glass',
 'fronts',
 'face',
 'inwards',
 'privacy',
 'high',
 'agenda',
 'em',
 'city',
 'home',
 'manyaryans',
 'muslims',
 'gangstas',
 'latinos',
 'christians',
 'italians',
 'irish',
 'moreso',
 'scuffles',
 'death',
 'stares',
 'dodgy',
 'dealings',
 'shady',
 'agreements',
 'never',
 'far',
 'awaybr',
 'br',
 'would',
 'say',
 'main',
 'appeal',
 'sho

In [9]:
def get_sentence_vector(tokens, model):
    word_vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(100)
    return np.mean(word_vectors, axis=0)

# Create sentence vectors
df['sentence_vector'] = df['tokens'].apply(lambda tokens: get_sentence_vector(tokens, model))
X = np.vstack(df['sentence_vector'])
y = df['sentiment']


In [10]:
X

array([[-0.0969433 ,  0.22745198,  0.07901271, ..., -0.21271417,
         0.02949088,  0.06550576],
       [-0.09839981,  0.24485652,  0.07585478, ..., -0.22020203,
         0.03818709,  0.07871734],
       [-0.10193736,  0.25436097,  0.07418124, ..., -0.2292589 ,
         0.02942798,  0.06645892],
       ...,
       [-0.10652266,  0.24578676,  0.07654767, ..., -0.22727303,
         0.02993019,  0.06718522],
       [-0.10872159,  0.2743416 ,  0.06014324, ..., -0.24951269,
         0.02846511,  0.05641394],
       [-0.10169419,  0.2508384 ,  0.07143973, ..., -0.22867884,
         0.02529067,  0.05852032]], dtype=float32)

In [11]:
X_trian,X_test,y_train ,y_test = train_test_split(X,y,random_state=42,test_size=0.2)

classifer = RandomForestClassifier(n_estimators=100,random_state=42)

classifer.fit(X_trian,y_train)



RandomForestClassifier(random_state=42)

In [13]:
# Predict sentiment
y_pred = classifer.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 68.61%


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize classifiers
classifiers = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'AdaBoost': AdaBoostClassifier(n_estimators=100, random_state=42),
    'Support Vector Classifier': SVC(kernel='linear', random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Neural Network': MLPClassifier(hidden_layer_sizes=(50,), max_iter=1000, random_state=42)
}

# Train and evaluate each classifier
results = {}
for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy

# Print the results
for name, accuracy in results.items():
    print(f"{name} Accuracy: {accuracy * 100:.2f}%")





Random Forest Accuracy: 68.61%
Gradient Boosting Accuracy: 67.26%
AdaBoost Accuracy: 69.06%
Support Vector Classifier Accuracy: 56.05%
Logistic Regression Accuracy: 61.88%
Neural Network Accuracy: 65.47%


In [15]:
model

In [16]:
model.save("Sentiment_analysis_model")